<a href="https://colab.research.google.com/github/DigitalTibetan/DigitalTibetan/blob/main/docs/computational_dharma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational dharma, really?

## A digital toolset to work with multiple language

This article just wants to give a quick overview of what's possible with today's natural language processing tools. All examples will focus on the Python programming language as an additional tool for working with Tibetan texts.

There are two ways to read this notebook:

- either treat it as a normal static website, and get an overview what current tools can do,
- click the button above `Open in Colab`. This will open this notebook in a Google colab instance (free), and will allow you to modify and execute the examples without installing any software locally. To execute a command in the notebook, press `<Shift><Enter>` and the current cell will be executed.

This will be just some first glimpse, more details and explanations will be added in later articles.

Let's start with some conversions between Wylie and Unicode Tibetan, using Esukhia's [pyetws](https://github.com/OpenPecha-dev/pyewts) that is a port of earlier work by [Roger Espel](https://github.com/rogerespel/ewts-js).

## Conversion between Wylie and Unicode Tibetan

In [15]:
%%capture
!pip install pyewts

In [4]:
import pyewts

In [11]:
converter=pyewts.pyewts()
print(converter.toUnicode("oM AHhUM:"))
print(converter.toWylie("སེམས་ཉིད་"))

ཨོཾ་ཨཱཿཧཱུཾ༔
sems nyid 


## Let's look at Sanskrit transliteration systems

In [ ]:
%%capture
!pip install indic_transliteration

In [13]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

In [17]:
transliterate("Kālacakra", sanscript.IAST, sanscript.DEVANAGARI)

'कालचक्र'